# Import Library

In [1]:
# Standard Imports
import numpy as np
import pandas as pd

# Additional Imports
import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

# Load Credentials and Create Movie Object

In [2]:
import json
with open('/Users/ericakitano/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['API-key'])

In [3]:
# Log-in to TMDB using my API-key
tmdb.API_KEY =  login['API-key']

In [4]:
## make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)

# Obtain Certification Data from `releases()`

## Define `get_movie_with_rating` function

In [7]:
def get_movie_with_rating(movie_id):
    
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    info=movie.info()
    
    releases = movie.releases()
    
    #Loop through countries in releases
    for c in releases['countries']:
        #if the country is US
        if c['iso_3166_1'] == 'US':
            # save a "certification" key in "info" with the certification
            info['certification'] = c['certification']
    return info


## Test `get_movie_with_rating` function

### Test#1: movie_id: tt0848228 ("The Avengers")

In [8]:
# Test get_movie_with_rating function with movie_id: tt0848228 ("The Avengers")
test = get_movie_with_rating("tt0848228")
test

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 220.336,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

Verified that `info` now contains `certification` key. 

### Test#2 : movie_id: tt0332280 ("The Notebook")

In [9]:
# Test get_movie_with_rating function with movie_id: tt0332280 ("The Notebook")
test2 = get_movie_with_rating("tt0332280")
test2

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 61.45,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/5ThIuO93vsk47oexKTSdfKEr7EC.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 260

Verified that the `get_movie_with_rating` function is working.

# Obtain Financial Data

### Designate a folder to save data obtained from API calls

In [10]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_basics.csv.gz', 'title_akas.csv.gz', 'title_ratings.csv.gz']

### Define `write_json` function

This function appends a list of records to a json file.

In [11]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

### Load in the Title Basics data

In [12]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv("Data/title_basics.csv.gz")

### Define a list of the Years to Extract from the API

Create Required Lists for the Outer Loop

In [13]:
YEARS_TO_GET = [2000,2001]

### Define an errors list

Create an empty errors list before the loops so that we can append and save the ids and error messages for any movie that causes an error. 

In [14]:
errors = [ ]